# How many cryptic PAS are supported by polyA tail containing reads?

Strategy for analysis:
- Predicted PAS from GTF file
- Subset quant GTF to cryptics
- pr.k_nearest on each interval (pick relaxed, allow overlaps + either direction)

Q1: What %age of events are supported by reads at given distance threshold?
- e.g. 0, 10, 25, 50, 100
- Each le_id can contain multiple 3'ends per event - first collapse to min distance as representative for each le_id
- Then apply range of cut-offs - number of events in each window


Q2: Are distinct isoforms supported for each event?
- e.g. STMN2s and ARHGAP32s of the cryptic world
- perform naive clustering/collapsing of pas by le_id e.g. 25/50nt 
  - How many PAS have multiple unique predicted isoforms?
  - How many have junction read support?




TO CONSIDER:

3-5 overhang could be missing some lower confidence/shorter supporting reads. Repeat counting of lower confidence PAS by running count_pas with BED file containing cryptic PAS from PAPA (all PAS for each le_id) AND/OR (e.g. separate runs) PAS clusters defined using the 3-5/6+ criteria (i.e. for the same clusters, is there additional support from short overhangs?)

Annotated events for comparison - first just use any evaluated annotated le_id (i.e. in any dataset)

In [5]:
import pyranges as pr
import pandas as pd
import numpy as np
from typing import Union, List, Optional, Tuple, Dict
import os

In [16]:
def nearest_threshold_wrapper(gr: pr.PyRanges,
                              gr2: pr.PyRanges,
                              thresholds: Union[int, List[int]],
                              id_col: str = "le_id",
                              nearest_only: bool = True,
                              return_grs: bool = False,
                              nearest_kwargs: dict = {"k": 10,
                                                      "strandedness": "same",
                                                      "overlap": True,
                                                      "how": None},
                              ) -> Tuple[Dict[str, set], Optional[Dict[str, pr.PyRanges]]]:
    '''Get events with nearest overlap passing a maximum distance threshold(s)

    Parameters
    ----------
    gr : pr.PyRanges
        _description_
    gr2 : pr.PyRanges
        _description_
    thresholds : Union[int, List[int]]
        _description_
    id_col : str, optional
        _description_, by default "le_id"
    nearest_only : bool, optional
        _description_, by default True
    return_grs : bool, optional
        _description_, by default False
    nearest_kwargs : _type_, optional
        _description_, by default {"k": 10, "strandedness": "same", "overlap": True, "how": None}

    Returns
    -------
    Tuple[Dict[str, set], Optional[Dict[str, pr.PyRanges]]]
        return {threshold: set(id_col values)} and optionally {threshold: pr.PyRanges} (if return_grs=True)
    '''
    

    if nearest_only:
        if "k" in nearest_kwargs.keys():
            kwargs = {key: v for key, v in nearest_kwargs.items()if key != "k"}
        gr_nr = gr.nearest(gr2, **kwargs)

    else:
        assert "k" in nearest_kwargs.keys()
        gr_nr = gr.k_nearest(gr2, **nearest_kwargs)


    # convert to absolute values (don't care if up/downstream rn)
    gr_nr = gr_nr.assign("DistanceAbs", lambda df: df.Distance.abs())
    
    # if nearest_only:
    #     # # for each le_id, select the smallest distance
    #     gr_nr = gr_nr.apply(lambda df: df.sort_values(by=[id_col, "DistanceAbs"]).drop_duplicates(subset=[id_col])).sort()

    if isinstance(thresholds, int):
        # subset to those pasing distance threshold
        pass_grs = {str(thresholds): gr_nr.subset(lambda df: df.DistanceAbs.le(thresholds))}

    else:
        pass_grs = {str(threshold): gr_nr.subset(lambda df: df.DistanceAbs.le(threshold)) for threshold in thresholds}

    # return set of ids at each threshold
    pass_ids = {threshold: set(gr.as_df()[id_col]) for threshold, gr in pass_grs.items()}

    if return_grs:
        return pass_ids, pass_grs
    else:
        return pass_ids


    


In [6]:
# BED files of polyA-tail containing reads - 3-5 nt overhang + 100 % As | 6+nt & >80 % As
# Pooling across all KD samples from all experiments
pas_all = pr.read_bed("../data/bulk_polya_reads/tdp_ko_collection/pas_clusters/condition__TDP43KD/two_class_simple/polya_clusters.bed")
# pooling x KD samples from i3Neuron datasets
pas_i3 = pr.read_bed("../data/bulk_polya_reads/tdp_ko_collection/pas_clusters/cell_type__i3_cortical___condition__TDP43KD/two_class_simple/polya_clusters.bed")
# pooling x KD samples from SH-SY5Y datasets
pas_shsy5y = pr.read_bed("../data/bulk_polya_reads/tdp_ko_collection/pas_clusters/cell_type__shsy5y___condition__TDP43KD/two_class_simple/polya_clusters.bed")
# poolin x KD samples from SK-N-BE2 datasets
pas_sknbe2 = pr.read_bed("../data/bulk_polya_reads/tdp_ko_collection/pas_clusters/cell_type__sknbe2___condition__TDP43KD/two_class_simple/polya_clusters.bed")

# GTF file of predicted last exons
quant_le = pr.read_gtf("../data/novel_ref_combined.last_exons.gtf")

# Table with cryptic le_ids (prior to mv filtering)
cryptics_df = pd.read_csv("../processed/2023-12-10_cryptics_summary_all_events.tsv", sep="\t")
# after manual validation (just to get some idea)
cryptics_mv_df = pd.read_csv("../processed/2023-12-10_cryptics_summary_all_events_bleedthrough_manual_validation.tsv", sep="\t")


In [31]:
# Combined differential analysis table for all events
dexseq_df = pd.read_csv("../processed/PAPA/2023-12-10_i3_cortical_zanovello.all_datasets.dexseq_apa.results.processed.cleaned.tsv", sep="\t")
dexseq_df

,experiment_name,binID,groupID,featureID,exonBaseMean,dispersion,stat,pvalue,padj,UsageCoefficient_base,...,annot_status,transcript_id,chromosome,strand,start,end,mean_PPAU_base,mean_PPAU_treatment,delta_PPAU_treatment_control,simple_event_type
0,brown_i3_cortical,ENSG00000021645.20:E001,ENSG00000021645.20,E001,32.627088,0.044220,90.962787,1.463997e-21,2.250563e-19,1.452069,...,novel,PAPA.TDP-4.18197.2,chr14,+,7.829867e+07,7.830620e+07,0.001914,0.033848,0.031934,spliced
1,brown_i3_cortical,ENSG00000021645.20:E002,ENSG00000021645.20,E002,622.521901,0.022117,135.739619,2.274890e-31,5.061630e-29,12.675156,...,annotated,"ENST00000557594.5,ENST00000428277.6,ENST000002...",chr14,+,7.986104e+39,7.986248e+39,0.998086,0.934776,-0.063310,spliced
2,brown_i3_cortical,ENSG00000021645.20:E002,ENSG00000021645.20,E002,622.521901,0.022117,135.739619,2.274890e-31,5.061630e-29,12.675156,...,annotated,"ENST00000557594.5,ENST00000428277.6,ENST000002...",chr14,+,7.986104e+39,7.986248e+39,0.998086,0.934776,-0.063310,spliced
3,brown_i3_cortical,ENSG00000048649.14:E001,ENSG00000048649.14,E001,163.262346,0.037855,88.319669,5.568698e-21,8.407740e-19,4.515264,...,novel,"PAPA.doxconc_DOX_0075_2.11065.7,PAPA.TDP43-G_S...",chr11,-,7.781315e+71,7.781379e+71,0.090158,0.462779,0.372622,spliced
4,brown_i3_cortical,ENSG00000048649.14:E001,ENSG00000048649.14,E001,163.262346,0.037855,88.319669,5.568698e-21,8.407740e-19,4.515264,...,annotated,"PAPA.doxconc_DOX_0075_2.11065.7,PAPA.TDP43-G_S...",chr11,-,7.781315e+71,7.781379e+71,0.090158,0.462779,0.372622,spliced
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109646,zanovello_skndz_curve_1,ENSG00000280789.2:E002,ENSG00000280789.2,E002,99.077638,12.000000,0.006587,9.353145e-01,9.785953e-01,5.982423,...,novel,PAPA.chx_tdp_CTRL_ctrl_4.15153.2,chr16,+,2.981955e+07,2.982252e+07,0.181220,0.333333,0.152113,distal_3utr_extension
109647,zanovello_skndz_curve_1,ENSG00000280832.2:E001,ENSG00000280832.2,E001,75.778902,0.099478,0.075875,7.829679e-01,9.189656e-01,6.014731,...,annotated,ENST00000626810.1,chr11,-,1.263502e+08,1.263518e+08,0.133144,0.147606,0.014462,spliced
109648,zanovello_skndz_curve_1,ENSG00000280832.2:E002,ENSG00000280832.2,E002,402.882205,0.085732,0.087526,7.673463e-01,9.134343e-01,12.145436,...,annotated,ENST00000629441.2,chr11,-,1.263409e+08,1.263423e+08,0.866856,0.852394,-0.014462,spliced
109649,zanovello_skndz_curve_1,ENSG00000282508.2:E001,ENSG00000282508.2,E001,43.093715,0.060733,0.017535,8.946533e-01,9.665471e-01,4.668928,...,annotated,ENST00000633109.1,chr19,-,2.112510e+05,2.117150e+05,0.678888,0.582547,-0.096341,spliced


In [33]:
# get lists of IDs for further analysis

# cryptic IDs
cryp_le_ids = set(cryptics_df.le_id)
cryp_le_ids_mv = set(cryptics_mv_df.le_id)

# annotated ids - any evaluated, non-cryptic annotated event
annot_any_le_ids = set(dexseq_df.loc[dexseq_df.annot_status == "annotated", "le_id"]) - set(cryp_le_ids)
print("num cryp ids - ", len(cryp_le_ids))
print("num man val cryp ids - ", len(cryp_le_ids_mv))
print("num annot, non-cryptic ids - ", len(annot_any_le_ids))

num cryp ids -  258
num man val cryp ids -  227
num annot, non-cryptic ids -  8529


In [7]:
# subset cryptic GTFs
quant_le_cryp = quant_le.subset(lambda df: df.le_id.isin(cryp_le_ids))
quant_le_cryp_mv = quant_le.subset(lambda df: df.le_id.isin(cryp_le_ids_mv))
quant_le_cryp_pas = quant_le_cryp.three_end()
quant_le_cryp_mv_pas = quant_le_cryp_mv.three_end()
quant_le_cryp_pas

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,15940476,15940477,.,+,.,PAPA.chx_tdp_DOX_ctrl_2.224,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000065526.12,SPEN,4.0,ENSG00000065526.12_4
1,chr1,.,exon,147623355,147623356,.,+,.,PAPA.doxconc_DOX_0075_2.1123,NaN,...,NaN,NaN,NaN,NaN,NaN,internal_exon_extension,ENSG00000116128.12,BCL9,2.0,ENSG00000116128.12_2
2,chr1,.,exon,147622692,147622693,.,+,.,PAPA.chx_tdp_DOX_ctrl_2.1186,NaN,...,NaN,NaN,NaN,NaN,NaN,internal_exon_extension,ENSG00000116128.12,BCL9,2.0,ENSG00000116128.12_2
3,chr1,.,exon,147623700,147623701,.,+,.,PAPA.doxconc_DOX_0075_1.1139,NaN,...,NaN,NaN,NaN,NaN,NaN,internal_exon_extension,ENSG00000116128.12,BCL9,2.0,ENSG00000116128.12_2
4,chr1,.,exon,21457149,21457150,.,+,.,PAPA.TDP-1.345,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000142794.19,NBPF3,3.0,ENSG00000142794.19_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970,chrX,.,exon,40653676,40653677,.,-,.,PAPA.TDP43_ctrl_3.26410,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000180182.11,MED14,3.0,ENSG00000180182.11_3
971,chrX,.,exon,131823775,131823776,.,-,.,PAPA.TDP43-F_S6.20505,NaN,...,NaN,NaN,NaN,NaN,NaN,internal_exon_extension,ENSG00000213468.7,FIRRE,1.0,ENSG00000213468.7_1
972,chrX,.,exon,131823777,131823778,.,-,.,PAPA.TDP43_ctrl_2.26896,NaN,...,NaN,NaN,NaN,NaN,NaN,internal_exon_extension,ENSG00000213468.7,FIRRE,1.0,ENSG00000213468.7_1
973,chrX,.,exon,108267641,108267642,.,-,.,ENSG00000197565.17,COL4A6,...,last,"108221374,108221374,108221374","108310747,108310747,108310747","ENST00000372216.8,ENST00000538570.5,ENST000006...","NULL,NULL,NULL",internal_exon_spliced,ENSG00000197565.17,COL4A6,1.0,ENSG00000197565.17_1


In [8]:
# remove junky columns
quant_le_cryp_pas = quant_le_cryp_pas[["le_id", "ref_gene_name", "ref_gene_id", "event_id"]]
quant_le_cryp_pas

,Chromosome,Start,End,Strand,le_id,ref_gene_name,ref_gene_id
0,chr1,15940476,15940477,+,ENSG00000065526.12_4,SPEN,ENSG00000065526.12
1,chr1,147623355,147623356,+,ENSG00000116128.12_2,BCL9,ENSG00000116128.12
2,chr1,147622692,147622693,+,ENSG00000116128.12_2,BCL9,ENSG00000116128.12
3,chr1,147623700,147623701,+,ENSG00000116128.12_2,BCL9,ENSG00000116128.12
4,chr1,21457149,21457150,+,ENSG00000142794.19_3,NBPF3,ENSG00000142794.19
...,...,...,...,...,...,...,...
970,chrX,40653676,40653677,-,ENSG00000180182.11_3,MED14,ENSG00000180182.11
971,chrX,131823775,131823776,-,ENSG00000213468.7_1,FIRRE,ENSG00000213468.7
972,chrX,131823777,131823778,-,ENSG00000213468.7_1,FIRRE,ENSG00000213468.7
973,chrX,108267641,108267642,-,ENSG00000197565.17_1,COL4A6,ENSG00000197565.17


In [9]:
# overlap with jnc reads from all PAS
cryp_pas_kn = quant_le_cryp_pas.k_nearest(pas_all, k=10, strandedness="same", overlap=True, how=None)
# convert to absolute values (don't care if up/downstream rn)
cryp_pas_kn = cryp_pas_kn.assign("DistanceAbs", lambda df: df.Distance.abs())

# for each le_id, select the smallest distance
cryp_pas_kn_nr = cryp_pas_kn.apply(lambda df: df.sort_values(by=["le_id", "DistanceAbs"]).drop_duplicates(subset=["le_id"])).sort()

# how 
n_cryp_supported = {str(d): cryp_pas_kn_nr.subset(lambda df: df.DistanceAbs.le(d)).le_id.nunique() for d in [0,10,25,50,100,200,500]}
print(n_cryp_supported)
{d: n / len(cryp_le_ids) for d,n in n_cryp_supported.items()}
# cryp_pas_kn_nr.as_df()


{'0': 54, '10': 112, '25': 122, '50': 131, '100': 138, '200': 151, '500': 176}


{'0': 0.20930232558139536,
 '10': 0.43410852713178294,
 '25': 0.4728682170542636,
 '50': 0.5077519379844961,
 '100': 0.5348837209302325,
 '200': 0.5852713178294574,
 '500': 0.6821705426356589}

In [10]:
# overlap with jnc reads from all PAS
cryp_pas_mv_kn = quant_le_cryp_pas.k_nearest(pas_all, k=10, strandedness="same", overlap=True, how=None)
# convert to absolute values (don't care if up/downstream rn)
cryp_pas_mv_kn = cryp_pas_mv_kn.assign("DistanceAbs", lambda df: df.Distance.abs())

# for each le_id, select the smallest distance
cryp_pas_mv_kn_nr = cryp_pas_mv_kn.apply(lambda df: df.sort_values(by=["le_id", "DistanceAbs"]).drop_duplicates(subset=["le_id"])).sort()

# how 
n_cryp_supported_mv = {str(d): cryp_pas_mv_kn_nr.subset(lambda df: df.DistanceAbs.le(d)).le_id.nunique() for d in [0,10,25,50,100,200,500]}
print(n_cryp_supported_mv)
{d: n / len(cryp_le_ids_mv) for d,n in n_cryp_supported_mv.items()}


{'0': 54, '10': 112, '25': 122, '50': 131, '100': 138, '200': 151, '500': 176}


{'0': 0.23788546255506607,
 '10': 0.4933920704845815,
 '25': 0.5374449339207048,
 '50': 0.5770925110132159,
 '100': 0.6079295154185022,
 '200': 0.6651982378854625,
 '500': 0.775330396475771}

For manually validated (i.e. bleedthrough IR artefacts removed):
~ 49 % +/- 10nt, rising ~ 61 % +/- 100nt

vs all initial cryptics:
~ 43 % +/- 10nt, rising ~ 53 % +/- 100nt

Haven't checked read support...

In [34]:
distance_thresholds = [0,10,25,50,100,200,500]

In [38]:

quant_le_annotany = quant_le.subset(lambda df: df.le_id.isin(annot_any_le_ids))
quant_le_annotany_pas = quant_le_annotany.three_end()[["le_id", "ref_gene_name", "ref_gene_id", "event_id"]]
quant_le_annotany_pas

,Chromosome,Start,End,Strand,le_id,ref_gene_name,ref_gene_id
0,chr1,24466377,24466378,+,ENSG00000001461.17_4,NIPAL3,ENSG00000001461.17
1,chr1,24472975,24472976,+,ENSG00000001461.17_5,NIPAL3,ENSG00000001461.17
2,chr1,24472975,24472976,+,ENSG00000001461.17_5,NIPAL3,ENSG00000001461.17
3,chr1,150160064,150160065,+,ENSG00000023902.14_2,PLEKHO1,ENSG00000023902.14
4,chr1,150159629,150159630,+,ENSG00000023902.14_2,PLEKHO1,ENSG00000023902.14
...,...,...,...,...,...,...,...
26586,chrY,13249855,13249856,-,ENSG00000183878.16_6,UTY,ENSG00000183878.16
26587,chrY,13249855,13249856,-,ENSG00000183878.16_6,UTY,ENSG00000183878.16
26588,chrY,13249855,13249856,-,ENSG00000183878.16_6,UTY,ENSG00000183878.16
26589,chrY,13249855,13249856,-,ENSG00000183878.16_6,UTY,ENSG00000183878.16


In [39]:
# get ids & grs where cryptics have pas junction support from any dataset across range of thresholds
cryp_pas_allpas_nr_ids, cryp_pas_allpas_nr_grs = nearest_threshold_wrapper(quant_le_cryp_pas, pas_all, distance_thresholds, nearest_only=False, return_grs=True)   
cryp_pasmv_allpas_nr_ids, cryp_pasmv_allpas_nr_grs = nearest_threshold_wrapper(quant_le_cryp_mv_pas, pas_all, distance_thresholds, nearest_only=False, return_grs=True)   

# repeat also for annotated, non-cryptic events
annot_pasmv_allpas_nr_ids, annot_pasmv_allpas_nr_grs = nearest_threshold_wrapper(quant_le_annotany_pas, pas_all, distance_thresholds, nearest_only=False, return_grs=True)   

In [40]:
print("Fraction of events that have junction read support at a given distance threshold")
print("All cryptics - ", {d: len(ids) / len(cryp_le_ids) for d,ids in cryp_pas_allpas_nr_ids.items()})
print("Manually validated cryptics - ", {d: len(ids) / len(cryp_le_ids_mv) for d,ids in cryp_pasmv_allpas_nr_ids.items()})
print("Annotated last exons - ", {d: len(ids) / len(annot_any_le_ids) for d, ids in annot_pasmv_allpas_nr_ids.items()})

Fraction of events that have junction read support at a given distance threshold
All cryptics -  {'0': 0.20930232558139536, '10': 0.43410852713178294, '25': 0.4728682170542636, '50': 0.5077519379844961, '100': 0.5348837209302325, '200': 0.5852713178294574, '500': 0.6821705426356589}
Manually validated cryptics -  {'0': 0.2290748898678414, '10': 0.46255506607929514, '25': 0.5022026431718062, '50': 0.5374449339207048, '100': 0.5550660792951542, '200': 0.6035242290748899, '500': 0.7136563876651982}
Annotated last exons -  {'0': 0.5229217962246453, '10': 0.617071168952984, '25': 0.6466174229100715, '50': 0.6610388087700786, '100': 0.6806190643686247, '200': 0.710517059444249, '500': 0.7603470512369562}
